### 검증용 추천 시스템 

In [3]:
import requests
from datetime import datetime as dt
from sklearn.preprocessing import MinMaxScaler 
import random
import json
import sys
import re
import numpy as np
import pandas as pd
sys.path.insert(0, '../../assets/modules')
now = pd.to_datetime(dt.today())
import json
# json.dump(data, indent="\t", ensure_ascii=False)
with open('../../../bepms/assets/json/Recommendation_json/question.json', 'r', encoding='utf-8') as f:
    question = json.load(f)
with open('../../../bepms/assets/json/Recommendation_json/GROUP_USER.json', 'r', encoding='utf-8') as g:
    group = json.load(g)
import copy
from callDB import DBHelper as hp
from ast import literal_eval
from truckEDA import EDAHelper
from dotenv import dotenv_values
server = "DEV"
config = dotenv_values("../../assets/modules/.env")

# callDB에 매개변수로 전달
f = hp(config, server)
eda = EDAHelper()

#ㅡㅡㅡㅡㅡㅡㅡㅡㅡ 개발서버 배포 된 post
# import requests
# import json

# url = "http://15.165.252.49:9090/api/web/main/recommend"

# payload = json.dumps({
#   "A_main": "기타",
#   "B_item": "담배",
#   "C_pkg": "박스",
#   "D_ton": "1",
#   "E_brand": "1",
#   "F_reliability_q1": "1",
#   "G_reliability_q2": "3",
#   "H_reliability_q3": "3",
#   "I_reliability_q4": "3",
#   "J_reliability_q5": "3",
#   "K_condition_q1": "5",
#   "L_condition_q2": "5",
#   "M_condition_q3": "1",
#   "N_condition_q4": "4",
#   "O_asset_q1": "1",
#   "P_asset_q2": "2",
#   "Q_asset_q3": "1"
# })
# headers = {
#   'Content-Type': 'application/json'
# }

# response = requests.request("POST", url, headers=headers, data=payload)

# print(response.text)
#ㅡㅡㅡㅡ 로컬 
url = "http://localhost:8086/api/web/main/recommend"

payload = json.dumps({
    "A_main": "기타",
    "B_item": "용달",
    "C_pkg": "톤(kg, g)",
    "D_ton": "1",
    "E_brand": "1",
    "F_reliability_q1": "1",
    "G_reliability_q2": "4",
    "H_reliability_q3": "5",
    "I_reliability_q4": "3",
    "J_reliability_q5": "3",
    "K_condition_q1": "5",
    "L_condition_q2": "5",
    "M_condition_q3": "3",
    "N_condition_q4": "4",
    "O_asset_q1": "1",
    "P_asset_q2": "2",
    "Q_asset_q3": "1"
})
# json.dump(data, payload, indent="\t", ensure_ascii=False)
headers = {
  'Content-Type': 'application/json'
}

api = requests.request("POST", url, headers=headers, data=payload).json()
where = api.get('where')
data = api.get('data')

quest = literal_eval(payload)
user_points = []
user_points.extend([int(quest.get('D_ton')), int(quest.get('E_brand')), int(quest.get('F_reliability_q1')), int(quest.get('G_reliability_q2')), int(quest.get('H_reliability_q3')), int(quest.get('I_reliability_q4')), int(quest.get('J_reliability_q5')), int(quest.get('K_condition_q1')), int(quest.get('L_condition_q2')), int(quest.get('M_condition_q3')), int(quest.get('N_condition_q4')), int(quest.get('O_asset_q1')), int(quest.get('P_asset_q2')), int(quest.get('Q_asset_q3'))])
user_points = ",".join(map(str, user_points))
user_points


def call_mycar():
    
    dfs = f.table(f'SELECT\
            m.MCR_ID,\
            m.MCR_SCD,\
            m.MCR_BRAND_CD,\
            m.MCR_TON,\
            m.MCR_YEAR_MM,\
            m.MCR_KM,\
            m.MCR_DESC,\
            m.MCR_PRICE,\
            m.MCR_PLATE_TYPE,\
            m.MCR_REAL_YN,\
            m.MCR_REG_DT,\
            m.MCR_DIRECT_YN,\
            m.MCR_AXLE,\
            m.MCR_WING_TOP,\
            m.MCR_IMG1,\
            m.MCR_IMG2,\
            m.MCR_IMG3,\
            m.MCR_IMG4,\
            m.MCR_IMG5,\
            m.MCR_IMG6,\
            m.MCR_IMG7,\
            m.MCR_IMG8,\
            m.MCR_IMG9,\
            m.MCR_IMG10,\
            m.MCR_IMG11,\
            m.MCR_IMG12,\
            m.MCR_IMG13,\
            m.MCR_IMG14,\
            m.MCR_IMG15,\
            m.MCR_IMG16,\
            m.MCR_IMG17,\
            m.MCR_IMG18,\
            m.MCR_IMG19,\
            m.MCR_IMG20,\
            d.DAL_ID,\
            d.DAL_COM_NUM,\
            d.DAL_REWARD_YN,\
            d.DAL_REG_DT,\
            d.DAL_LAST_LOGIN_DT,\
            c.CLICK_NUM \
        FROM\
            TB_MYCAR AS m\
        LEFT OUTER JOIN TB_DEALER AS d\
        ON\
            m.DAL_ID = d.DAL_ID\
        LEFT OUTER JOIN TB_CLICK AS c\
        ON\
            m.MCR_ID = c.PROD_ID\
        LEFT OUTER JOIN TB_MYCAR_OPTION AS op\
        ON\
            m.MCR_ID = op.MCR_ID {where};')

    DEALER_SALES_CNT = f.table(f'SELECT\
                m.DAL_ID,\
                COUNT(*) AS DAL_TOTAL_UPT_CNT\
            FROM\
                TB_MYCAR AS m\
            LEFT OUTER JOIN TB_DEALER AS d\
            ON\
                m.DAL_ID = d.DAL_ID\
            LEFT OUTER JOIN TB_CLICK AS c\
            ON\
                m.MCR_ID = c.PROD_ID\
            LEFT OUTER JOIN TB_MYCAR_OPTION AS op\
            ON\
                m.MCR_ID = op.MCR_ID \
            {where}\
            GROUP BY\
                m.DAL_ID;')

    df = pd.merge(dfs, DEALER_SALES_CNT, on='DAL_ID', how='left')
    return df

def preprocess_mycar():
    df = call_mycar()
    df['DESC_LEN'] = df['MCR_DESC']
    df['DAL_REG_DT'] = df['DAL_REG_DT'].astype(str).str.replace(pat=r'[a-zA-Z]+', repl= r' ', regex=True)
    df['DAL_LAST_LOGIN_DT'] = df['DAL_LAST_LOGIN_DT'].astype(str).str.replace(pat=r'[a-zA-Z]+', repl= r' ', regex=True)
    df['MCR_REG_DT'] = df['MCR_REG_DT'].astype(str).str.replace(pat=r'[a-zA-Z]+', repl= r' ', regex=True)

    df["DESC_LEN"] = df["DESC_LEN"].str.replace(pat=r'[^\w]', repl=r'', regex=True)
    df["DESC_LEN"] = df["DESC_LEN"].str.len().fillna(0)

    df['DAL_LAST_LOGIN_DT'] = pd.to_datetime(df['DAL_LAST_LOGIN_DT'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
    df['DAL_REG_DT'] = pd.to_datetime(df['DAL_REG_DT'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
    df['MCR_REG_DT'] = pd.to_datetime(df['MCR_REG_DT'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

    df["DAL_COM_NUM"] = df["DAL_COM_NUM"].astype(str).str.len()

    df.loc[df["DAL_COM_NUM"] <= 5, 'DAL_COM_NUM'] = 0
    df.loc[df["DAL_COM_NUM"] > 5, 'DAL_COM_NUM'] = 1


    for i in range(1, 21):
        df[['MCR_IMG{}'.format(i)]] = df[['MCR_IMG{}'.format(i)]].where(df[['MCR_IMG{}'.format(i)]].isnull(), 1).fillna(0)


    df['IMG_COUNT'] = 0

    for i in range(1, 21):
        df['IMG_COUNT'] += df[f'MCR_IMG{i}']
        df.drop(columns=[f"MCR_IMG{i}"], inplace=True)

    df['MCR_YEAR_MM'] = pd.to_datetime(df['MCR_YEAR_MM'], format='%Y-%m-%d', errors='coerce')
    df.drop(df[df['MCR_YEAR_MM'] > now].index, inplace=True)

    df['MCR_PLATE_TYPE'] = df['MCR_PLATE_TYPE'].replace('0402', 1).replace('0401', 0)

    df['MCR_REAL_YN'] = df['MCR_REAL_YN'].replace('N', 0).replace('Y', 1)
    df['MCR_DIRECT_YN'] = df['MCR_DIRECT_YN'].replace('N', 0).replace('Y', 1)

    df['DAL_REWARD_YN'] = df['DAL_REWARD_YN'].replace('N', 0).replace('Y', 1)

    df['DAL_LAST_LOGIN_DT'].fillna(df['DAL_REG_DT'], inplace=True)

    df['MCR_TON'] = df['MCR_TON'].replace('기타', '0').astype(float)

    df['MCR_KM'] = df['MCR_KM'].astype("int64")
    df['MCR_PRICE'] = df['MCR_PRICE'].astype("int64")

    df['DAILYPRODUCT'] = (df['DAL_LAST_LOGIN_DT'] - df['DAL_REG_DT']).dt.days / df['DAL_TOTAL_UPT_CNT']
    df['DAILYVIEW'] = (df['CLICK_NUM'] / ((now - df['DAL_REG_DT']).dt.days + 1)).fillna(0)
    df['DAILYVIEW'] = -df['DAILYVIEW']

    df = df.drop(columns=[ 'DAL_REG_DT', 'DAL_REG_DT', 'DAL_LAST_LOGIN_DT', 'CLICK_NUM', 'DAL_ID', 'DAL_TOTAL_UPT_CNT'])

    df = df.drop(df[df['MCR_KM'] / ((now - df['MCR_YEAR_MM']).dt.days) > 600].index)

    df['MCR_KM'] = -df['MCR_KM']


    df = df.drop(df[df['MCR_PRICE'] <= 100].index)
    df = df.drop(df[df['MCR_PRICE'] >= 30000].index)

    df.set_index('MCR_ID', inplace=True)

    return df

def get_car_bias_points():
    df = preprocess_mycar()
    user_points1 = user_points_d(user_points)
    scaler = MinMaxScaler()
    
    car_ton = [
            (1, 1.9),
            (2, 3.5),
            (4.5, 7.9),
            (8, 18),
            (19, 27),
            ('트랙터'),
        ]

    car_brand = [
        ('C'),
        ('D')
    ]

    # 신뢰도 높음 점수
    df_reliability = df[[
        "MCR_REAL_YN", # 실매물확인여부
        "DAL_COM_NUM", # 종사원증번호여부
        "DAILYPRODUCT", # 일평균등록매물
        "DAL_REWARD_YN", # 헛걸음보상제공여부
    ]]
    reliability = scaler.fit_transform(df_reliability)

    # 차량상태 좋음 점수
    df_condition = df[[
        "MCR_KM", # 주행거리
        "MCR_PRICE", # 가격
        "DESC_LEN", # 상세설명 길이
        "IMG_COUNT", # 이미지 수
    ]]

    condition = scaler.fit_transform(df_condition)
    condition = np.concatenate((condition, scaler.fit_transform(df[["MCR_YEAR_MM"]])), axis=1)
    # 보유자산 낮음 점수
    df_asset = df[[
        "MCR_DIRECT_YN", # 직거래 매물여부
        "MCR_PLATE_TYPE", # 번호판 종류
        "DAILYVIEW", # 일평균조회수
    ]]

    asset = scaler.fit_transform(df_asset)

    #톤 점수
    # if user_points1['TON']['TON_Q1'] != '6' :
    if (user_points1['TON']['TON_Q1'] != '6'):
        dfa = df.copy()
        dfa['톤수_점수'] = None
        for i, t in enumerate(car_ton[0:5]):
            dfa.loc[
                (dfa['MCR_TON'] >= t[0]) & (dfa['MCR_TON'] <= t[1]), '톤수_점수'
            ] = (len(car_ton[0:5]) / (len(car_ton[0:5]) * (len(car_ton[0:5]) - 1))) * ((i + 1) - 1)
        ton = scaler.fit_transform(pd.DataFrame(-abs(dfa['톤수_점수'] - int(user_points1['TON']['TON_Q1']))))

    # 브랜드 점수
    dfa_2 = df.copy()
    dfa_2['브랜드_점수'] = None
    for i, b in enumerate(car_brand):
        globals()["i_temp"] = i
        k = dfa_2.loc[
            (dfa_2['MCR_BRAND_CD'].str.contains(b)), '브랜드_점수'
        ] = (len(car_brand) / (len(car_brand) * (len(car_brand) - 1))) * ((i + 1) - 1) 
        l = dfa_2['브랜드_점수']
    brand = scaler.fit_transform(pd.DataFrame(-abs(dfa_2['브랜드_점수'] - int(user_points1['BRAND']['BRAND_Q1']))))

    if (user_points1['TON']['TON_Q1'] != '6'):
        df_knn = pd.DataFrame({
            'RELIABILITY': (reliability.sum(axis=1) / reliability.shape[1]),
            'CONDITION': (condition.sum(axis=1) / condition.shape[1]),
            'ASSET': (asset.sum(axis=1) / asset.shape[1]),
            'TON': (ton.sum(axis=1) / ton.shape[1]),
            'BRAND' : (brand.sum(axis=1) / brand.shape[1]),
        }, index=df.index,)
    else : 
        df_knn = pd.DataFrame({
            'RELIABILITY': (reliability.sum(axis=1) / reliability.shape[1]),
            'CONDITION': (condition.sum(axis=1) / condition.shape[1]),
            'ASSET': (asset.sum(axis=1) / asset.shape[1]),
            'BRAND' : (brand.sum(axis=1) / brand.shape[1]),
        }, index=df.index,)

    return df_knn

def user_points_d(user_points):
    questions = question
    user_points=user_points.replace(",","")
    question_user = copy.deepcopy(questions)

    question_user['TON']['TON_Q1']['QUEST']['QUESTION_NUMBER'] = user_points[0]
    question_user['BRAND']['BRAND_Q1']['QUEST']['QUESTION_NUMBER'] = user_points[1]
    question_user['RELIABILITY']['RELIABILITY_Q1']['QUEST']['QUESTION_NUMBER'] = user_points[2]
    question_user['RELIABILITY']['RELIABILITY_Q2']['QUEST']['QUESTION_NUMBER'] = user_points[3]
    question_user['RELIABILITY']['RELIABILITY_Q3']['QUEST']['QUESTION_NUMBER'] = user_points[4]
    question_user['RELIABILITY']['RELIABILITY_Q4']['QUEST']['QUESTION_NUMBER'] = user_points[5]
    question_user['RELIABILITY']['RELIABILITY_Q5']['QUEST']['QUESTION_NUMBER'] = user_points[6]
    question_user['CONDITION']['CONDITION_Q1']['QUEST']['QUESTION_NUMBER'] = user_points[7]
    question_user['CONDITION']['CONDITION_Q2']['QUEST']['QUESTION_NUMBER'] = user_points[8]
    question_user['CONDITION']['CONDITION_Q3']['QUEST']['QUESTION_NUMBER'] = user_points[9]
    question_user['CONDITION']['CONDITION_Q4']['QUEST']['QUESTION_NUMBER'] = user_points[10]
    question_user['ASSET']['ASSET_Q1']['QUEST']['QUESTION_NUMBER'] = user_points[11]
    question_user['ASSET']['ASSET_Q2']['QUEST']['QUESTION_NUMBER'] = user_points[12]
    question_user['ASSET']['ASSET_Q3']['QUEST']['QUESTION_NUMBER'] = user_points[13]

    user_points1 = dict() 
    for axis, quests in question_user.items():
        user_points1[axis] = dict()
        for q_num, quest in quests.items():
            user_points1[axis][q_num] = quest['QUEST']["QUESTION_NUMBER"]

#     user_points = dict() # 이거는 파라미터로 받아야 됨
#     for axis, quests in questions.items():
#         user_points[axis] = dict()
#         for q_num, quest in quests.items():
#             user_points[axis][q_num] = random.randint(1, quest["QUESTION_NUMBER"])
            
    return user_points1

def get_user_bias_points():
    questions = question
    questions['TON']['TON_Q1']['QUEST']['QUESTION_NUMBER'] = ["1", "2", "3", "4", "5"]
    user_points1 = user_points_d(user_points)
    
    user_bias_points = dict()
    for (axis_question, quests_question), (axis_answer, quests_answer) in zip(questions.items(), user_points1.items()):
        user_bias_points[axis_answer] = list()
        for (q_num, quest_question), (a_num, quest_answer) in zip(quests_question.items(), quests_answer.items()):
            user_bias_points[axis_answer].append((len(quest_question['QUEST']["QUESTION_NUMBER"]) / (len(quest_question['QUEST']["QUESTION_NUMBER"]) * (len(quest_question['QUEST']["QUESTION_NUMBER"]) - 1))) * (int(quest_answer) - 1))
        user_bias_points[axis_answer] = sum(user_bias_points[axis_answer]) / len(user_bias_points[axis_answer])
        
    return user_bias_points

def sort_by_distance():
    df_knn = get_car_bias_points()
    user_bias_points = get_user_bias_points()
    if ("TON" not in df_knn) :
        X, Y, Z, B = np.array(df_knn['RELIABILITY']), np.array(df_knn['CONDITION']), np.array(df_knn['ASSET']), np.array(df_knn['BRAND'])

        distance = np.sqrt(
                    ((X - user_bias_points['RELIABILITY']) ** 2) +
                    ((Y - user_bias_points['CONDITION']) ** 2) +
                    ((Z - user_bias_points['ASSET']) ** 2) +
                    ((B - user_bias_points['BRAND']) ** 2)
                )
        df_knn['거리'] = distance
        result_ID = df_knn.sort_values(by='거리').index[:6]

    elif (df_knn['TON'].isnull().values.all() == True) :
        df_knn = df_knn.dropna(how='all', axis='columns')
        X, Y, Z, B = np.array(df_knn['RELIABILITY']), np.array(df_knn['CONDITION']), np.array(df_knn['ASSET']), np.array(df_knn['BRAND'])

        distance = np.sqrt(
                    ((X - user_bias_points['RELIABILITY']) ** 2) +
                    ((Y - user_bias_points['CONDITION']) ** 2) +
                    ((Z - user_bias_points['ASSET']) ** 2) +
                    ((B - user_bias_points['BRAND']) ** 2)
                )
        df_knn['거리'] = distance
        result_ID = df_knn.sort_values(by='거리').index[:6]

    else :
        X, Y, Z, T, B = np.array(df_knn['RELIABILITY']), np.array(df_knn['CONDITION']), np.array(df_knn['ASSET']), np.array(df_knn['TON']), np.array(df_knn['BRAND'])

        distance = np.sqrt(
                    ((X - user_bias_points['RELIABILITY']) ** 2) +
                    ((Y - user_bias_points['CONDITION']) ** 2) +
                    ((Z - user_bias_points['ASSET']) ** 2) +
                    ((T - user_bias_points['TON']) ** 2) +
                    ((B - user_bias_points['BRAND']) ** 2)
                )
        df_knn['거리'] = distance
        result_ID = df_knn.sort_values(by='거리').index[:6]

    return result_ID

def main():
    try :
        result_ID = sort_by_distance()
    except ValueError:
         result_ID = ''
    recommend = re.sub(' ', '', str(list(result_ID))[1:-1])
        
    return recommend
            

def predict():
    recommend = main()
    pre = recommend.split(',')
    pre = list(map(int, pre))
    return pre

def f1_score():
    try:
        pre = predict()
        user_bias_points = get_user_bias_points()
        df_knn = get_car_bias_points()

        if 'TON' in df_knn.columns:
            df_knn_a = df_knn[(df_knn['TON'] >= user_bias_points['TON'] - 0.2) & (df_knn['TON']<= user_bias_points['TON'] + 0.2) & (df_knn['BRAND'] >= user_bias_points['BRAND'] - 0.3) & (df_knn['BRAND']<= user_bias_points['BRAND'] + 0.3) & (df_knn['RELIABILITY'] >= user_bias_points['RELIABILITY'] - 0.4) & (df_knn['RELIABILITY']<= user_bias_points['RELIABILITY'] + 0.4) & (df_knn['CONDITION'] >= user_bias_points['CONDITION'] - 0.4) & (df_knn['CONDITION']<= user_bias_points['CONDITION'] + 0.4) & (df_knn['ASSET'] >= user_bias_points['ASSET'] - 0.4) & (df_knn['ASSET']<= user_bias_points['ASSET'] + 0.4)]
        else:
            df_knn_a = df_knn[(df_knn['BRAND'] >= user_bias_points['BRAND'] - 0.5) & (df_knn['BRAND']<= user_bias_points['BRAND'] + 0.5) & (df_knn['RELIABILITY'] >= user_bias_points['RELIABILITY'] - 0.3) & (df_knn['RELIABILITY']<= user_bias_points['RELIABILITY'] + 0.3) & (df_knn['CONDITION'] >= user_bias_points['CONDITION'] - 0.3) & (df_knn['CONDITION']<= user_bias_points['CONDITION'] + 0.3) & (df_knn['ASSET'] >= user_bias_points['ASSET'] - 0.33) & (df_knn['ASSET']<= user_bias_points['ASSET'] + 0.33)]

        if len(df_knn_a) > 6:
            df_knn_a = df_knn[(df_knn['TON'] >= user_bias_points['TON'] - 0.2) & (df_knn['TON']<= user_bias_points['TON'] + 0.2) & (df_knn['BRAND'] >= user_bias_points['BRAND'] - 0.3) & (df_knn['BRAND']<= user_bias_points['BRAND'] + 0.3) & (df_knn['RELIABILITY'] >= user_bias_points['RELIABILITY'] - 0.33) & (df_knn['RELIABILITY']<= user_bias_points['RELIABILITY'] + 0.33) & (df_knn['CONDITION'] >= user_bias_points['CONDITION'] - 0.33) & (df_knn['CONDITION']<= user_bias_points['CONDITION'] + 0.33) & (df_knn['ASSET'] >= user_bias_points['ASSET'] - 0.33) & (df_knn['ASSET']<= user_bias_points['ASSET'] + 0.33)]
        else :
            df_knn_a = df_knn[(df_knn['TON'] >= user_bias_points['TON'] - 0.5) & (df_knn['TON']<= user_bias_points['TON'] + 0.5) & (df_knn['BRAND'] >= user_bias_points['BRAND'] - 0.3) & (df_knn['BRAND']<= user_bias_points['BRAND'] + 0.3) & (df_knn['RELIABILITY'] >= user_bias_points['RELIABILITY'] - 0.5) & (df_knn['RELIABILITY']<= user_bias_points['RELIABILITY'] + 0.5) & (df_knn['CONDITION'] >= user_bias_points['CONDITION'] - 0.5) & (df_knn['CONDITION']<= user_bias_points['CONDITION'] + 0.5) & (df_knn['ASSET'] >= user_bias_points['ASSET'] - 0.5) & (df_knn['ASSET']<= user_bias_points['ASSET'] + 0.5)]   

        if len(df_knn_a) < 6:
            df_knn_a = df_knn[(df_knn['TON'] >= user_bias_points['TON'] - 0.2) & (df_knn['TON']<= user_bias_points['TON'] + 0.2) & (df_knn['BRAND'] >= user_bias_points['BRAND'] - 0.3) & (df_knn['BRAND']<= user_bias_points['BRAND'] + 0.3) & (df_knn['RELIABILITY'] >= user_bias_points['RELIABILITY'] - 0.5) & (df_knn['RELIABILITY']<= user_bias_points['RELIABILITY'] + 0.5) & (df_knn['CONDITION'] >= user_bias_points['CONDITION'] - 0.5) & (df_knn['CONDITION']<= user_bias_points['CONDITION'] + 0.5) & (df_knn['ASSET'] >= user_bias_points['ASSET'] - 0.5) & (df_knn['ASSET']<= user_bias_points['ASSET'] + 0.5)]

        if df_knn_a.empty:
            df_knn_a = df_knn[(df_knn['TON'] >= user_bias_points['TON'] - 0.5) & (df_knn['TON']<= user_bias_points['TON'] + 0.5) & (df_knn['BRAND'] >= user_bias_points['BRAND'] - 0.5) & (df_knn['BRAND']<= user_bias_points['BRAND'] + 0.5) & (df_knn['RELIABILITY'] >= user_bias_points['RELIABILITY'] - 0.55) & (df_knn['RELIABILITY']<= user_bias_points['RELIABILITY'] + 0.55) & (df_knn['CONDITION'] >= user_bias_points['CONDITION'] - 0.55) & (df_knn['CONDITION']<= user_bias_points['CONDITION'] + 0.55) & (df_knn['ASSET'] >= user_bias_points['ASSET'] - 0.57) & (df_knn['ASSET']<= user_bias_points['ASSET'] + 0.584)]   
        else : 
            df_knn_a = df_knn_a

        if len(df_knn) < 6:
            df_knn_a = df_knn

        X, Y, Z, T, B = np.array(df_knn_a['RELIABILITY']), np.array(df_knn_a['CONDITION']), np.array(df_knn_a['ASSET']), np.array(df_knn_a['TON']), np.array(df_knn_a['BRAND'])
        distance_a = np.sqrt(
                    ((X - user_bias_points['RELIABILITY']) ** 2) +
                    ((Y - user_bias_points['CONDITION']) ** 2) +
                    ((Z - user_bias_points['ASSET']) ** 2) +
                    ((T - user_bias_points['TON']) ** 2) +
                    ((B - user_bias_points['BRAND']) ** 2)
                )
        df_knn_a['거리'] = distance_a
        result = df_knn_a.sort_values(by='거리').index[:6]

        anw = re.sub(' ', '', str(list(result))[1:-1])
        anw = anw.split(',')
        anw = list(map(int, anw))
    
    except:
        anw = []
        pre = []

    if len(anw) != 0:
        usr_anw = 0
        for idx, val in enumerate(pre):
            if val in anw:
                usr_anw += 1
            else : 
                continue
        precision = usr_anw / len(pre)

        re_anw = 0
        for r_idx, r_val in enumerate(anw):
            if r_val in pre:
                re_anw += 1
            else : 
                continue
        recall = re_anw / len(anw)
        
        f1_score = 2.0 / (1/precision + 1/recall)

    else :
        f1_score = 0

    return f1_score


### f1 Score

In [4]:
f1_score()

c:\users\joowo\appdata\local\programs\python\python38\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


0.8333333333333334